In [5]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn.functional import softmax
import transformers
from transformers import AutoModel, BertTokenizerFast

# ГПУ для більших швидких обчислень
device = torch.device("cuda")

In [6]:
data = pd.read_csv("../input/sarcasm-detection/sarcasm_detection.csv")
data = data[['headline', 'is_sarcastic']]
data.head()

,headline,is_sarcastic
0,thirtysomething scientists unveil doomsday clo...,1
1,dem rep. totally nails why congress is falling...,0
2,eat your veggies: 9 deliciously different recipes,0
3,inclement weather prevents liar from getting t...,1
4,mother comes pretty close to using word 'strea...,1


In [7]:
data['is_sarcastic'].value_counts(normalize = True)

is_sarcastic
0    0.541679
1    0.458321
Name: proportion, dtype: float64

Бачимо, що значного дізбалансу в даних нема. Тому можемо не добавляти ваги для класів

In [8]:
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import re
batch_size = 32

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

class SarcasmDataset(Dataset):
    '''
    Класс сімейства Датасет для наших даних 
    '''
    def __init__(self, data, transform = None):
        '''
        Конструктор
        '''
        self.data = data
        self.transform = transform
        
        
    def __len__(self):
        return len(self.data)

    
    def __getitem__(self, idx):

        item = self.data.iloc[idx]
        
        target = int(item['is_sarcastic'])
        sentence = item.headline
        'Токенайзуємо речення та екстрактимо токени зі масками'
        tokens = tokenizer.encode_plus(
            sentence,
            add_special_tokens = True,
            max_length=64,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        

        seq = tokens['input_ids']
        mask = tokens['attention_mask']
        
        if self.transform:
            seq = self.transform(seq)
        
        'Повертаємо токени зі масками речення, та класс, до якого належить оброблене речення'
        return {
            'input_ids': seq,
            'attention_mask': mask,
            'class': target
         }
    
dataset = SarcasmDataset(data) #Ініцюалізуємо
trainset, validset = random_split(dataset, [0.7, 0.3]) #Ділимо на train та valid
#Та ініцюалізуємо train та test-дані 
train_loader = DataLoader(trainset, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(validset, shuffle=True, batch_size=batch_size)

In [9]:
class BERT_Arch(nn.Module):

    def __init__(self):
        super(BERT_Arch, self).__init__()
        
        self.bert = AutoModel.from_pretrained('bert-base-uncased')
      
        self.fc1 = nn.Sequential(nn.Dropout(0.1),
                                nn.Linear(768,1))

    #define the forward pass
    def forward(self, sent_id, mask):
        
        #Берт Модель 
        _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)
        #І поверх неї модель класифікац

        
        # output layer
        x = self.fc1(cls_hs)

        return x

In [12]:
from transformers import AdamW
import tqdm
from torchmetrics.classification import BinaryAccuracy
model = BERT_Arch().to(dtype=torch.float16, device='cuda')
# define the optimizer
loss_f = torch.nn.BCEWithLogitsLoss()
optimizer = AdamW(model.parameters(),lr = 1e-5) 
val_loss_g = [1]
metrics = BinaryAccuracy().to(device)
i = 1;

while True:
    print(f"Epoch {i}")
    # Train loop
    train_tqdm = tqdm.tqdm(train_loader)
    model.train()
    for batch in train_tqdm:
        input_ids, attention_mask, cls = batch['input_ids'], batch['attention_mask'], batch['class']

        # Передаємо до ГПУ наші тензори'
        input_ids, attention_mask, cls = input_ids.to(device), attention_mask.to(device), cls.to(device)
        #Онуляємо градієнти 
        optimizer.zero_grad()
        model.zero_grad()
        
        #Предиктимо класи
        output = model(input_ids.squeeze(1), attention_mask.squeeze(1)).squeeze(1)
        
        # Рахуємо трати
        L = loss_f(output, cls.float())
        
        acc = accuracy(softmax(output), cls)
        #Передаємо до моделі градієнт втрат
        L.backward()
        
        
        optimizer.step() #Оновлення параметрів отпимізатору
        train_tqdm.set_description(f"Train loss: {L.item()}  Accuracy -- {acc}") #Вивід у термінал значення втрат
        train_tqdm.refresh()
        
    valid_tqdm = tqdm.tqdm(valid_loader)    
    model.eval()
    
    with torch.no_grad():
        val_loss = []
        acc = []
        for batch in valid_tqdm:
            input_ids, attention_mask, cls = batch['input_ids'], batch['attention_mask'], batch['class']

            input_ids, attention_mask, cls = input_ids.to(device), attention_mask.to(device), cls.to(device)
            
            output = model(input_ids.squeeze(1), attention_mask.squeeze(1)).squeeze(1)

            L = loss_f(output, cls.float())
            val_loss.append(L.item())
            
            acc.append(metrics(output, cls).cpu())
            
            
            valid_tqdm.set_description(f"Val loss: {L.item()}")
            valid_tqdm.refresh()
            
    val_loss_g.append(np.mean(val_loss))
    
    if abs(val_loss_g[-2] - val_loss_g[-1]) > 0.0001:
        print(f"Epoch {i}, Val loss: {np.mean(val_loss)}")
        print(f"Epoch {i}, Accuracy: {np.mean(acc)}")
        i += 1
    else:
        break
    

Epoch 1


  0%|          | 0/1211 [00:00<?, ?it/s]/tmp/ipykernel_34/2639436552.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  acc = accuracy(softmax(output), cls)
Train loss: 0.7043365836143494  Accuracy -- 0.4000000059604645: 100%|██████████| 1211/1211 [03:48<00:00,  5.29it/s]
Val loss: 0.703502357006073: 100%|██████████| 519/519 [00:35<00:00, 14.63it/s] 


Epoch 1, Val loss: 0.6896028604810638
Epoch 1, Accuracy: 0.5413984060287476
Epoch 2


Train loss: 0.6468445062637329  Accuracy -- 0.699999988079071: 100%|██████████| 1211/1211 [03:49<00:00,  5.29it/s]
Val loss: 0.7129405736923218: 100%|██████████| 519/519 [00:35<00:00, 14.63it/s]


Epoch 2, Val loss: 0.6898658112524102
Epoch 2, Accuracy: 0.5413710474967957
Epoch 3


Train loss: 0.6604858636856079  Accuracy -- 0.4000000059604645: 100%|██████████| 1211/1211 [03:48<00:00,  5.29it/s]
Val loss: 0.6830333471298218: 100%|██████████| 519/519 [00:35<00:00, 14.65it/s]
